<a href="https://colab.research.google.com/github/u-masao/vector-search-tutorial-for-japanese/blob/main/Session04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語のためのベクトル検索ハンズオン：Session 4

このセッションでは、Qdrant の実践的な使い方を確認します。

Docker で Qdrant を実行して、Python SDK で基本的な操作をします。

## 環境構築


### Google Colab 向け Docker インストール

In [1]:
# Copyright 2024 Drengskapur
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# @title {display-mode:"form"}
# @markdown <br/><br/><center><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/docker.svg" height="150"><img src="https://cdn.jsdelivr.net/gh/drengskapur/docker-in-colab/assets/colab.svg" height="150"></center><br/>
# @markdown <center><h1>Docker in Colab</h1></center><center>github.com/drengskapur/docker-in-colab<br/><br/><br/><b>udocker("run hello-world")</b></center><br/>
def udocker_init():
    import os
    if not os.path.exists("/home/user"):
        !pip install udocker > /dev/null
        !udocker --allow-root install > /dev/null
        !useradd -m user > /dev/null
    print(f'Docker-in-Colab 1.1.0\n')
    print(f'Usage:     udocker("--help")')
    print(f'Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples')

    def execute(command: str):
        user_prompt = "\033[1;32muser@pc\033[0m"
        print(f"{user_prompt}$ udocker {command}")
        !su - user -c "udocker $command"

    return execute

udocker = udocker_init()

Docker-in-Colab 1.1.0

Usage:     udocker("--help")
Examples:  https://github.com/indigo-dc/udocker?tab=readme-ov-file#examples


In [2]:
# udocker("pull qdrant/qdrant")

In [3]:
udocker("run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant")

# 初回実行時は正常に終了しないことがあります。しばらくして応答がなければランタイムをリセットしてください。

user@pc$ udocker run -p 127.0.0.1:6333:6333 -p 127.0.0.1:6334:6334 qdrant/qdrant
 
 ****************************************************************************** 
 *                                                                            * 
 *               STARTING 129078ea-1602-3d16-a1a5-c25c8ac482bf                * 
 *                                                                            * 
 ****************************************************************************** 
 executing: entrypoint.sh
           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Version: 1.9.0, build: b99d5074
Access web UI at http://localhost:6333/dashboard

2024-04-23T01:27:18.682879Z  INFO storage::content_manager::consensus::persistent: Initializing new raft state at ./storage/raft_state.json    
2024-04-23T01:27:19.178790Z  INFO qdrant: Distrib

###  SageMaker 向け docker 実行

!docker run -p 127.0.0.1:6333:6333 qdrant/qdranbt


## パッケージのインストールと読み込み

In [4]:
!pip install -q qdrant-client sentence-transformers datasets

In [5]:
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import pandas as pd
import numpy as np

## Qdrant の利用

以下を実行します。

- ベクトル作成
  - データ取得
  - テキスト作成
  - 埋め込み計算
- Qdrant Client を初期化
- コレクションを初期化(Cosine計算モード)
- ポイント(アイテム)を登録
  - ベクトル、ペイロード、インデックス
- ベクトル検索のクエリ例
- ベクトル検索&フィルタリングのクエリ例

### データの読み込みと埋め込みベクトルの計算

In [6]:
# Hugging Face からデータセットを取得
# データセットの説明 → https://huggingface.co/datasets/sbintuitions/JMTEB
dataset = load_dataset("sbintuitions/JMTEB", name="livedoor_news")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for sbintuitions/JMTEB contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/sbintuitions/JMTEB
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  wa

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 5163
    })
    validation: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1106
    })
    test: Dataset({
        features: ['url', 'timestamp', 'title', 'text', 'label'],
        num_rows: 1107
    })
})

In [8]:
dataset['train'].features

{'url': Value(dtype='string', id=None),
 'timestamp': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news'], id=None)}

In [9]:
# データセットからデータフレームを作成(100件をサンプリング)
text_df  = pd.DataFrame(dataset['train']).sample(100, random_state = 12345)  # 100件をランダムに抽出

# ラベル名を作成
label_names = dataset['train'].features['label'].names
text_df['label_name'] = text_df['label'].apply(lambda x: label_names[x])

# URL 順に表示
text_df.sort_values('url')

,url,timestamp,title,text,label,label_name
949,http://news.livedoor.com/article/detail/4457415/,2009-11-18T15:00:00+0900,“婚活成功メイク”で素敵な結婚を,\nこの頃では料理婚活、ゴルフ婚活、アウトドア婚活など様々な工夫をこらした婚活が行われており...,5,peachy
4584,http://news.livedoor.com/article/detail/4564175/,2010-01-22T19:50:00+0900,上質＆カジュアルディナーでお父さんにありがとうを,\n横浜ロイヤルパークホテルB1 カフェ「カフェ フローラ」に登場のバレンタインディナー。こ...,5,peachy
2826,http://news.livedoor.com/article/detail/4669665/,2010-03-20T08:00:00+0900,【Sports Watch】ダルビッシュが衝撃の告白、新球種を生み出した,\nテレビ朝日「報道ステーション」（19日放送分）では、野球解説者の栗山英樹氏が開幕戦直前と...,7,sports-watch
4747,http://news.livedoor.com/article/detail/4703658/,2010-04-07T07:30:00+0900,【Sports Watch】岡田監督が胸中を告白「3試合ともキツイ」,\n「岡田さん、NEWS ZEROってしってます？」という北澤氏に、岡田監督は「俺、テレビほ...,7,sports-watch
511,http://news.livedoor.com/article/detail/4782525/,2010-05-21T15:20:00+0900,インタビュー：御影倫代「仕事が好きだから、家でも仕事の話がしたい！」,\n——今回は御影さんに、「グレイズ・アナトミー」シーズン1の1話、2話、3話をご覧になって...,5,peachy
...,...,...,...,...,...,...
191,http://news.livedoor.com/article/detail/6827489/,2012-08-06T10:00:00+0900,ドラクエXで遊ぶのに必要な物 PCの周辺機器を流用できる【デジ通】,\n有名タイトルのオンライン化はFF（ファイナル・ファンタジー）14で大コケしたために賛否両...,1,it-life-hack
2844,http://news.livedoor.com/article/detail/6834780/,2012-08-08T10:00:00+0900,廃プレイせずとも楽しめるドラクエX！ プレイしなくても他プレーヤーとの差が広がらない仕組み...,\n同時に大人数がゲーム世界に参加して遊ぶことになるが、遊んでいる時間が長ければ長いほどゲー...,1,it-life-hack
3355,http://news.livedoor.com/article/detail/6879979/,2012-08-23T06:55:00+0900,ソフトバンク、AQUOS PHONE Xx 106SHにブラウザが強制終了す不具合でソフトウ...,\nソフトバンクモバイルは21日、今夏モデルのAndroid 4.0（開発コード名：IceC...,6,smax
2351,http://news.livedoor.com/article/detail/6887027/,2012-08-24T20:55:00+0900,NTTドコモ、GALAXY Tab 10.1 LTE SC-01DにAndroid 4.0 ...,\nNTTドコモは24日、次世代高速データ通信規格LTEによるサービス「Xi（クロッシィ）」...,6,smax


In [10]:
# 埋め込み用のテキストを作成する関数
def build_concat_text(row):

    # テンプレート
    return f'''
        # タイトル

        {row['title']}

        ## ニュースカテゴリ

        {row['label_name']}

        ## 記事

        {row['text']}
    '''

# 埋め込み対象テキストを作成
text_df['embed_text'] = text_df.apply(build_concat_text, axis=1)

# ライセンス表記の行を削除
text_df = text_df[~text_df['url'].str.startswith('このディレクトリにあるすべての記事ファイル')]

# URL からアイテムIDを計算
text_df['id'] = text_df['url'].replace('.*detail/([0-9]*)/',r'\1',regex=True).astype(int)

# タイトルの長さを計算(フィルタリング用)
text_df['title_length'] = text_df['title'].str.len()

# 記事本体の長さを計算(フィルタリング用)
text_df['text_length'] = text_df['text'].str.len()

# URL 順で表示
text_df.sort_values('id')

,url,timestamp,title,text,label,label_name,embed_text,id,title_length,text_length
949,http://news.livedoor.com/article/detail/4457415/,2009-11-18T15:00:00+0900,“婚活成功メイク”で素敵な結婚を,\nこの頃では料理婚活、ゴルフ婚活、アウトドア婚活など様々な工夫をこらした婚活が行われており...,5,peachy,\n # タイトル\n\n “婚活成功メイク”で素敵な結婚を\n...,4457415,16,752
4584,http://news.livedoor.com/article/detail/4564175/,2010-01-22T19:50:00+0900,上質＆カジュアルディナーでお父さんにありがとうを,\n横浜ロイヤルパークホテルB1 カフェ「カフェ フローラ」に登場のバレンタインディナー。こ...,5,peachy,\n # タイトル\n\n 上質＆カジュアルディナーでお父さんに...,4564175,24,775
2826,http://news.livedoor.com/article/detail/4669665/,2010-03-20T08:00:00+0900,【Sports Watch】ダルビッシュが衝撃の告白、新球種を生み出した,\nテレビ朝日「報道ステーション」（19日放送分）では、野球解説者の栗山英樹氏が開幕戦直前と...,7,sports-watch,\n # タイトル\n\n 【Sports Watch】ダルビッ...,4669665,36,928
4747,http://news.livedoor.com/article/detail/4703658/,2010-04-07T07:30:00+0900,【Sports Watch】岡田監督が胸中を告白「3試合ともキツイ」,\n「岡田さん、NEWS ZEROってしってます？」という北澤氏に、岡田監督は「俺、テレビほ...,7,sports-watch,\n # タイトル\n\n 【Sports Watch】岡田監督...,4703658,34,671
511,http://news.livedoor.com/article/detail/4782525/,2010-05-21T15:20:00+0900,インタビュー：御影倫代「仕事が好きだから、家でも仕事の話がしたい！」,\n——今回は御影さんに、「グレイズ・アナトミー」シーズン1の1話、2話、3話をご覧になって...,5,peachy,\n # タイトル\n\n インタビュー：御影倫代「仕事が好きだ...,4782525,34,1317
...,...,...,...,...,...,...,...,...,...,...
191,http://news.livedoor.com/article/detail/6827489/,2012-08-06T10:00:00+0900,ドラクエXで遊ぶのに必要な物 PCの周辺機器を流用できる【デジ通】,\n有名タイトルのオンライン化はFF（ファイナル・ファンタジー）14で大コケしたために賛否両...,1,it-life-hack,\n # タイトル\n\n ドラクエXで遊ぶのに必要な物 PCの...,6827489,33,2289
2844,http://news.livedoor.com/article/detail/6834780/,2012-08-08T10:00:00+0900,廃プレイせずとも楽しめるドラクエX！ プレイしなくても他プレーヤーとの差が広がらない仕組み...,\n同時に大人数がゲーム世界に参加して遊ぶことになるが、遊んでいる時間が長ければ長いほどゲー...,1,it-life-hack,\n # タイトル\n\n 廃プレイせずとも楽しめるドラクエX！...,6834780,51,1553
3355,http://news.livedoor.com/article/detail/6879979/,2012-08-23T06:55:00+0900,ソフトバンク、AQUOS PHONE Xx 106SHにブラウザが強制終了す不具合でソフトウ...,\nソフトバンクモバイルは21日、今夏モデルのAndroid 4.0（開発コード名：IceC...,6,smax,\n # タイトル\n\n ソフトバンク、AQUOS PHONE...,6879979,55,1414
2351,http://news.livedoor.com/article/detail/6887027/,2012-08-24T20:55:00+0900,NTTドコモ、GALAXY Tab 10.1 LTE SC-01DにAndroid 4.0 ...,\nNTTドコモは24日、次世代高速データ通信規格LTEによるサービス「Xi（クロッシィ）」...,6,smax,\n # タイトル\n\n NTTドコモ、GALAXY Tab ...,6887027,78,2371


In [11]:
# 統計情報を表示
text_df.describe()

,label,id,title_length,text_length
count,100.00000,1.000000e+02,100.000000,100.00000
mean,3.94000,6.124563e+06,37.340000,1082.16000
std,2.80627,6.321838e+05,15.799386,681.26292
min,0.00000,4.457415e+06,9.000000,232.00000
25%,1.00000,5.835135e+06,27.000000,590.00000
50%,4.00000,6.281853e+06,35.000000,867.00000
75%,7.00000,6.601976e+06,44.250000,1457.50000
max,8.00000,6.888284e+06,87.000000,4615.00000


In [43]:
# 埋め込みモデルを初期化
#   日本語の埋め込みで評判が良いモデルを利用
#   https://huggingface.co/intfloat/multilingual-e5-small
model = SentenceTransformer("intfloat/multilingual-e5-large")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [44]:
# 埋め込みモデルの出力次元酢を取得
embedding_dim = model.get_sentence_embedding_dimension()
print("Embedding dimension:", embedding_dim)

Embedding dimension: 1024


In [45]:
# タイトルと本文を結合したテキストをベクトル化
# multilingual-e5-small では最初の512トークン分までを反映
vectors = model.encode(text_df['embed_text'].values, normalize_embeddings=False)

In [46]:
# 埋め込みベクトルをファイルに保存
np.save("vectors.npy", vectors)

In [47]:
# 埋め込みベクトルをロード
# vectors = np.load("vectors.npy")

In [48]:
# 取得した埋め込みベクトルの概要
print("ndarray.shape:",vectors.shape)
print("各ベクトルの長さ(L2ノルム)の算術平均:",np.linalg.norm(vectors, ord=2, axis=1).mean())

ndarray.shape: (100, 1024)
各ベクトルの長さ(L2ノルム)の算術平均: 1.0


In [18]:
# サンプル表示
vectors[0]

array([ 0.01647516,  0.01339592, -0.03885332, -0.03085563,  0.05582241,
       -0.01911408,  0.03018196,  0.01156899,  0.03257352,  0.04047669,
        0.07249547, -0.01850226,  0.04363635,  0.00389617, -0.03058336,
        0.04904106,  0.06614374, -0.06298248, -0.03575558, -0.05058193,
        0.05792959,  0.01371834, -0.02630949,  0.0489661 ,  0.05782246,
        0.02430588, -0.00374517,  0.0705405 ,  0.03388409, -0.03659912,
       -0.05619214, -0.05489861,  0.03863645, -0.05840165,  0.10084745,
        0.0204209 , -0.05373559,  0.00298681,  0.04263383, -0.06494065,
       -0.08489714,  0.05871398,  0.0424255 ,  0.06615069,  0.01504599,
        0.1119694 , -0.05665908,  0.05371187, -0.05953417, -0.02511076,
        0.00574738,  0.10704706, -0.01389739,  0.08452754,  0.02019988,
       -0.02082599, -0.03049351, -0.07392799, -0.0910313 ,  0.01983926,
        0.08405369,  0.0216017 ,  0.00057762,  0.01489909,  0.11248856,
        0.03922565, -0.00280502,  0.03425073, -0.04668858,  0.01

## Qdrant Client を初期化

In [19]:
from qdrant_client import QdrantClient

qdrant_url = "http://localhost:6333"
qdrant_collection_name = "livedoor_news"

# Qdrant Client を初期化
client = QdrantClient(url=qdrant_url)

In [20]:
# コレクションの情報を取得
# status = client.get_collection( collection_name=qdrant_collection_name )
# print(status)

# コレクションを削除
# status = client.delete_collection( collection_name=qdrant_collection_name )
# print(status)

In [21]:
from qdrant_client.models import Distance, VectorParams

# コレクション(テーブル)を初期化
# ベクトル次元 384次元、距離: cosine
# (既にコレクションが定義されている場合はエラーが出ます)
#  -> b'{"status":{"error":"Wrong input: Collection `livedoor_news` already exists!"},"time":0.0000995}'
client.create_collection(
    collection_name=qdrant_collection_name,
    vectors_config=VectorParams(size=embedding_dim, distance=Distance.COSINE),
)

True

In [22]:
# ペイロードのインデックスのスキーマを定義
# None の場合はインデックスを作成しない
payload_index_field_types = {
    "id":"integer",  # URL から切り出した ID
    "url":None,  # 記事の URL
    "timestamp":"datetime",  # タイムスタンプ
    "label_name":"keyword",  # 記事区分名
    "label":"integer",  # 記事区分ID
    "text_length":"integer",  # テキストの文字列長
}

In [23]:
from qdrant_client.models import PointStruct, Batch

# ポイント(アイテム)を追加
# ベクトルとペイロードを追加
operation_info = client.upsert(
    collection_name=qdrant_collection_name,
    wait=True,  # 検索可能な状態になるまで応答を待つオプション
    points=Batch(
        ids=text_df['id'].to_list(),
        payloads=text_df[payload_index_field_types.keys()].to_dict('records'),
        vectors=vectors,
    )
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [24]:
# Qdrant の Payload にインデックスを設定
for field_name, field_type in payload_index_field_types.items():
    if field_type is None:
        continue
    print(f"field_name: {field_name}, field_type: {field_type}")
    client.create_payload_index(
        collection_name=qdrant_collection_name,
        field_name=field_name,
        field_schema=field_type,
    )

field_name: id, field_type: integer
field_name: timestamp, field_type: datetime
field_name: label_name, field_type: keyword
field_name: label, field_type: integer
field_name: text_length, field_type: integer


## 検索

In [25]:
# クエリ文字列からクエリベクトルを作成
query_text = 'インフラ関連の整備'
query_vector = model.encode(query_text)

In [26]:
# クエリベクトルの形
query_vector.shape

(384,)

In [27]:
# クエリベクトルに対するベクトル検索(Cosine類似度)
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    with_payload=True, # 応答にペイロードを含める
    limit=5
)

_ = [print(x) for x in search_result]

id=6704860 version=0 score=0.83389235 payload={'id': 6704860, 'label': 6, 'label_name': 'smax', 'text_length': 1721, 'timestamp': '2012-06-29T06:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6704860/'} vector=None shard_key=None
id=6528524 version=0 score=0.82203406 payload={'id': 6528524, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 915, 'timestamp': '2012-05-04T11:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6528524/'} vector=None shard_key=None
id=6564883 version=0 score=0.8211835 payload={'id': 6564883, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 851, 'timestamp': '2012-05-16T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6564883/'} vector=None shard_key=None
id=6701563 version=0 score=0.8206021 payload={'id': 6701563, 'label': 6, 'label_name': 'smax', 'text_length': 1840, 'timestamp': '2012-06-28T10:25:00+0900', 'url': 'http://news.livedoor.com/article/detail/6701563/'} vector=None shard_key=None
id=6879979 

In [28]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[FieldCondition(key="label_name", match=MatchValue(value="it-life-hack"))]  # ペイロードの文字列一致でフィルタリング
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=5,
)

_ = [print(x) for x in search_result]

id=6564883 version=0 score=0.8211835 payload={'id': 6564883, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 851, 'timestamp': '2012-05-16T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6564883/'} vector=None shard_key=None
id=6624672 version=0 score=0.8166499 payload={'id': 6624672, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1288, 'timestamp': '2012-06-04T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6624672/'} vector=None shard_key=None
id=6668966 version=0 score=0.81526744 payload={'id': 6668966, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 435, 'timestamp': '2012-06-18T15:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6668966/'} vector=None shard_key=None
id=6739559 version=0 score=0.81511456 payload={'id': 6739559, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1704, 'timestamp': '2012-07-09T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6739559/'} vector=None shard_key=N

In [29]:
from qdrant_client.models import Filter, FieldCondition, Range

# ベクトル検索 & フィルタリングの検索例
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(
                key="text_length",
                range=Range(  # ペイロードの値の範囲でフィルタリング
                    gte=600,
                    lte=1000,
                ),
            ),
        ]
    ),
    with_payload=True, # 応答にペイロードを含める
    limit=10,
)

_ = [print(x) for x in search_result]

id=6528524 version=0 score=0.82203406 payload={'id': 6528524, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 915, 'timestamp': '2012-05-04T11:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6528524/'} vector=None shard_key=None
id=6564883 version=0 score=0.8211835 payload={'id': 6564883, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 851, 'timestamp': '2012-05-16T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6564883/'} vector=None shard_key=None
id=6231231 version=0 score=0.8005925 payload={'id': 6231231, 'label': 2, 'label_name': 'kaden-channel', 'text_length': 669, 'timestamp': '2012-01-29T11:30:00+0900', 'url': 'http://news.livedoor.com/article/detail/6231231/'} vector=None shard_key=None
id=6273148 version=0 score=0.7983458 payload={'id': 6273148, 'label': 4, 'label_name': 'movie-enter', 'text_length': 704, 'timestamp': '2012-02-12T22:55:00+0900', 'url': 'http://news.livedoor.com/article/detail/6273148/'} vector=None shard_key=Non

In [30]:

from qdrant_client.models import Filter, FieldCondition, MatchValue

# ベクトル検索 & フィルタリングの検索例
# 2つのフィルタリング条件を And で結合
search_result = client.search(
    collection_name=qdrant_collection_name,
    query_vector=query_vector,  # 検索キーベクトルを指定
    query_filter=Filter(
        must=[
            FieldCondition(  # ペイロードの文字列一致でフィルタリング
                key="label_name",
                match=MatchValue(value="it-life-hack")
            ),
            FieldCondition(  # ペイロードの値の範囲でフィルタリング
                key="text_length",
                range=Range( gte=600, lte=1700,
                ),
            ),
        ]
    ),
    with_payload=True,  # ペイロードを取得
    limit=5,
)

_ = [print(x) for x in search_result]

id=6564883 version=0 score=0.8211835 payload={'id': 6564883, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 851, 'timestamp': '2012-05-16T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6564883/'} vector=None shard_key=None
id=6624672 version=0 score=0.8166499 payload={'id': 6624672, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1288, 'timestamp': '2012-06-04T17:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6624672/'} vector=None shard_key=None
id=6568107 version=0 score=0.81475824 payload={'id': 6568107, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1282, 'timestamp': '2012-05-17T13:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6568107/'} vector=None shard_key=None
id=6834780 version=0 score=0.7983947 payload={'id': 6834780, 'label': 1, 'label_name': 'it-life-hack', 'text_length': 1553, 'timestamp': '2012-08-08T10:00:00+0900', 'url': 'http://news.livedoor.com/article/detail/6834780/'} vector=None shard_key=N